In [2]:
import matplotlib.pyplot as plt
import numpy as np

from qutip import * 
import os

# Matplotlib Einstellungen gemäß den LaTeX-Caption-Formatierungen
plt.rcParams.update({
    'text.usetex': True,              # Enable LaTeX for text rendering
    'font.family': 'serif',           # Use a serif font family
    'font.serif': 'Palatino',         # Set Palatino as the serif font
    'text.latex.preamble': r'\usepackage{amsmath}',
    'font.size': 20,                   # Font size for general text
    'axes.titlesize': 20,              # Font size for axis titles
    'axes.labelsize': 20,              # Font size for axis labels
    'xtick.labelsize': 20,             # Font size for x-axis tick labels
    'ytick.labelsize': 20,             # Font size for y-axis tick labels
    'legend.fontsize': 20,             # Font size for legends
    'figure.figsize': [8, 6],          # Size of the plot (width x height)
    'figure.autolayout': True,         # Automatic layout adjustment
    'savefig.format': 'svg',           # Default format for saving figures
    'figure.facecolor': 'none',        # Make the figure face color transparent
    'axes.facecolor': 'none',          # Make the axes face color transparent
    'savefig.transparent': True        # Save figures with transparent background
})

output_dir = r"C:\Users\leopo\OneDrive - UT Cloud\Uni\Semester_9\Master_thesis\Figures_From_Python"
os.makedirs(output_dir, exist_ok=True)

In [3]:
# base 2 lvl atom
psi0 = basis(2,0) # Vector |0>
psi1 = basis(2,1) # Vector |1>

In [4]:
# Set the system parameters
N       = 1      # number of atoms
n_chains = 1     # number of chains
omega_0 = 1.     # Energy splitting of every atom
omega   = 1.     # driving

J_x, J_y, J_z = 0., 0., 0.  # Coupling between components of (neighboring) spins in all dimensions
# with periodic boundary
J_flipflop = 1.  # Flip-flop interaction of (neighboring) spins

# initial state
state_list = [psi1] + [psi0] * (N - 1)
psiini = tensor(state_list) # Vector |100...>

In [5]:
# Setup N-Qubit operators
sx_list, sy_list, sz_list, sm_list, sp_list = [], [], [], [], []
for i in range(N):
    op_list = [qeye(2)] * N
    op_list[i] = sigmax()
    sx_list.append(tensor(op_list))
    op_list[i] = sigmay()
    sy_list.append(tensor(op_list))
    op_list[i] = sigmaz()
    sz_list.append(tensor(op_list))
    op_list[i] = sigmam()
    sm_list.append(tensor(op_list))
    op_list[i] = sigmap()
    sp_list.append(tensor(op_list))

In [6]:
# Interaction coefficients (2D)
Jx, Jy, Jz = np.zeros((N, N)), np.zeros((N, N)), np.zeros((N, N))

''' # Simple chain 
# Set flip-flop interaction terms (nearest neighbors with periodic boundary)
for i in range(N):
    Jff[i, (i + 1) % N] = J_flipflop  # Interaction with the next spin
    Jff[(i + 1) % N, i] = J_flipflop  # Interaction with the previous spin (symmetric)   
    Jx[i, (i + 1) % N] = J_x  
    Jx[(i + 1) % N, i] = J_x
    Jy[i, (i + 1) % N] = J_y
    Jy[(i + 1) % N, i] = J_y
    Jz[i, (i + 1) % N] = J_z  
    Jz[(i + 1) % N, i] = J_z
'''

# Cylindrical arrangement of n_chains or length n
n = N // n_chains

def Jff_cylindrical(N, n_chains, J_flipflop=1.0, inter_chain_strength=1.0):
    # Number of atoms per chain
    n = N // n_chains
    if N % n_chains != 0:
        raise ValueError("N must be divisible by n_chains.")

    # Initialize the Jff interaction matrix
    Jff = np.zeros((N, N))

    # Intra-chain interactions (block diagonal)
    for chain in range(n_chains):
        for i in range(n):
            # Map local indices to global indices
            global_i = chain * n + i
            global_j = chain * n + (i + 1) % n  # Periodic within the chain

            # Add flip-flop interaction within the chain
            Jff[global_i, global_j] = J_flipflop
            Jff[global_j, global_i] = J_flipflop

    # Inter-chain interactions (nearest-neighbor chains with periodic boundary)
    for chain in range(n_chains):
        for i in range(n):
            # Map local indices to global indices for two adjacent chains
            global_i_chain1 = chain * n + i
            global_i_chain2 = ((chain + 1) % n_chains) * n + i  # Modulo n_chains for periodic boundary

            # Add identity-like connections between the chains
            Jff[global_i_chain1, global_i_chain2] = inter_chain_strength
            Jff[global_i_chain2, global_i_chain1] = inter_chain_strength

    return Jff


$$$\displaystyle H = \sum_{n=1}^N \left[ \frac{\omega_0}{2} \sigma_z^{(n)} + \omega \sigma_x^{(n)} \right] 
- \frac{1}{2} \sum_n^{N-1} [ J_x^{(n)} \sigma_x^{(n)} \sigma_x^{(n+1)} + J_y^{(n)} \sigma_y^{(n)} \sigma_y^{(n+1)} + J_z^{(n)} \sigma_z^{(n)} \sigma_z^{(n+1)}]
-\sum_{n=1}^N J_{ff}^{(n)} \sigma_+^{(n)} \sigma_-^{(n+1 \mod N)}.
$$

$$
\displaystyle H = \sum_{n=1}^N \left[ \frac{\omega_0}{2} \sigma_z^{(n)} + \omega \sigma_x^{(n)} \right] 
- J_{ff} \sum_{n=1}^N \sigma_+^{(n)} \sigma_-^{(n+1 \mod N)}.
$$

In [7]:
# define the Hamiltonian
def hamiltonian(omega, omega0, N):
    H = 0

    # Add energy splitting terms (diagonal in h)
    for i in range(N):
        H -= 0.5 * omega_0 * sz_list[i] + omega * sx_list[i]

    # Add interaction terms for Jx, Jy, Jz
    for i in range(N):
        for j in range(N):
            H -= 0.5 * Jx[i, j] * sx_list[i] * sx_list[j]
            H -= 0.5 * Jy[i, j] * sy_list[i] * sy_list[j]
            H -= 0.5 * Jz[i, j] * sz_list[i] * sz_list[j]
            H -= Jff[i, j] * (sp_list[i] * sm_list[j] + sm_list[i] * sp_list[j])     # Add flip-flop interaction terms
    
    return H

H = hamiltonian()

In [8]:
nbar = 1.0 # average photon (excitation) number

# collective decay and dephasing
gamma_dephase_collective = 0.1
gamma_decay_collective = 0.1 * nbar

# Define the collective decay collapse operator
c_ops_decay   = [np.sqrt(gamma_decay_collective)   * sum(sp_list[i] for i in range(N))] # Define the collective lowering operator (sigmam) as a sum over all spins
c_ops_dephase = [np.sqrt(gamma_dephase_collective) * sum(sz_list[i] for i in range(N))]

'''
gamma_decay = 0.1 * np.ones(N)  # Example decay rate for each spin
# Collapse operators for unique decay and dephasing
c_ops_decay = [np.sqrt(gamma_decay[i] * nbar) * 0.5 * (sx_list[i] - 1j * sy_list[i]) for i in range(N)] # this is sigmam()
gamma_dephase = 0.1 * np.ones(N)
c_ops_dephase = [np.sqrt(gamma_dephase[i]) * sz_list[i] for i in range(N)]

'''

# Combine dephasing and decay
c_ops = c_ops_decay + c_ops_dephase 

In [9]:
times = np.linspace(0, 30, 1000) # list of times
# evolution
result = mesolve(H, psiini, times, c_ops, sz_list, options={"store_states": True})
# Plot the expecation value
for i in range(N):
    plt.plot(times, result.expect[i], label=fr"${i}$", linestyle='-')  # Solid line

plt.legend(title="$i$")
plt.xlabel("Time"), plt.ylabel(r"$\langle \sigma_z^{{{i}}} \rangle$")
plt.title("Dynamics of spin chain with qubit dephasing");
plt.show()

In [10]:
# Projection operators for each spin onto the upper state |1>
projector_upper = [tensor([psi1 * psi1.dag() if i == j else qeye(2) for j in range(N)]) for i in range(N)]
excitation_operator = sum(sm_list[i] * sp_list[i] for i in range(N)) / 1  # \sum_i sigma_-^i * sigma_+^i

# evolution
result = mesolve(H, psiini, times, c_ops, projector_upper + [excitation_operator])
# Plot the population of the upper state for each spin
for i in range(N):
    plt.plot(times, result.expect[i], label=fr"${i}$", linestyle='-')  # Solid line
plt.plot(times, result.expect[N],label=r"$\langle n \rangle$", linestyle='--')

plt.legend(title="$i$")
plt.xlabel("Time")
plt.ylabel(r"Population P($|1\rangle_i$)")
plt.title("Dynamics of spin chain with qubit dephasing and decay")
plt.show()